# MNIST with ANN

### Imports

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import confusion_matrix

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [6]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms